In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [5]:
df = pd.read_csv('data/2022-04-01T12_df_sales_detail.csv')
df['date'] = pd.to_datetime(df['date'])

df['total_price'] = df.groupby(['sale_id', 'user_id'])['price'].transform("sum")

In [6]:
df.head(3)

,sale_id,good,price,date,user_id,total_price
0,1000001,mexican pizza,720,2022-02-04 10:00:24,1c1543,720
1,1000002,chefs pizza,840,2022-02-04 10:02:28,a9a6e8,930
2,1000002,orange juice,90,2022-02-04 10:02:28,a9a6e8,930


In [7]:
mean_price = df['price'].mean()
print(f'Средняя цена товара: {mean_price:.2f}')

Средняя цена товара: 590.28


In [8]:
daily_sales = df.groupby([df['date'].dt.date])['total_price'].sum().reset_index()
daily_sales.columns = ['Date', 'Total Sales']

In [9]:
top_goods_revenue = df.groupby('good')['price'].sum().sort_values(ascending=False).head(5)
top_goods_revenue

good
chefs pizza               24558240
double pepperoni pizza    22558380
chicken bbq pizza         17622540
farmhouse pizza           16612440
greek pizza               15976080
Name: price, dtype: int64

Динамика ежедневных продаж:

In [10]:
fig_daily_sales = px.line(
    daily_sales,
    x='Date',
    y='Total Sales',
    title='Динамика ежедневных продаж',
    labels={'Date': 'Дата', 'Total Sales': 'Общий объем продаж'},
    template='plotly_dark'
)
fig_daily_sales.update_traces(line=dict(width=2)) 
fig_daily_sales.show()

Топ-5 популярных товаров по суммарной выручке: 

In [11]:
fig_top_goods = px.bar(
    top_goods_revenue.reset_index(),
    x='price',
    y='good',
    orientation='h',
    color_discrete_sequence=['darkblue'],
    title='Топ-5 товаров по выручке',
    labels={'price': 'Общая выручка', 'good': 'Товар'}
)
fig_top_goods.update_layout(barmode='stack', yaxis={'categoryorder':'total ascending'})
fig_top_goods.show()

In [12]:
sales = pd.read_csv('data/2022-04-01T12_df_sales.csv')
web_logs = pd.read_csv('data/2022-04-01T12_df_web_logs.csv')

In [13]:
web_logs.describe()

,load_time
count,1.964507e+06
mean,7.396452e+01
std,1.107026e+02
min,0.000000e+00
25%,6.180000e+01
50%,7.000000e+01
75%,7.810000e+01
max,4.197500e+03


Cредний срок, который проходит от захода пользователя на сайт до совершения покупки. Будем считать, что заход на сайт относится к покупке, если он был совершён не ранее, чем за два часа до совершения покупки:

In [14]:
sales["date"] = pd.to_datetime(sales["date"])
web_logs["date"]  = pd.to_datetime(web_logs["date"])

web_logs = web_logs.sort_values(["user_id", "date"])
sales = sales.sort_values(["user_id", "date"])

merged = sales[["sale_id", "user_id", "date"]] \
    .rename(columns={"date": "sale_time"}) \
    .merge(web_logs[["user_id", "date"]].rename(columns={"date": "visit_time"}),
        on="user_id", how="left",
    )

window_start = merged["sale_time"] - pd.Timedelta(hours=2)
in_window = (merged["visit_time"] >= window_start) \
      & (merged["visit_time"] <= merged["sale_time"])

tmp = merged[in_window]

first_visit = tmp.groupby("sale_id", as_index=False)["visit_time"].min()

res = sales[["sale_id", "date"]].rename(columns={"date": "sale_time"}) \
    .merge(first_visit, on="sale_id", how="inner")
res["delta_minutes"] = (res["sale_time"] - res["visit_time"]).dt.total_seconds() / 60

avg_minutes = res["delta_minutes"].mean()

print(round(avg_minutes))

17


In [15]:
sales["day"] = sales["date"].dt.date
min_day = sales["day"].min()
max_day = sales["day"].max()
days = pd.date_range(min_day, max_day, freq="D").date
print(len(days))

57


In [ ]:
from useful_functions import get_revenue_all

rows = []
daily_user_dfs = {} 

for d in days:
    df_users = get_revenue_all(sales, web_logs, d, d + pd.Timedelta(days=1))
    daily_user_dfs[d] = df_users 

    rows.append({
        "day": d,
        "avg_revenue_per_user": df_users["total_revenue"].mean() if len(df_users) else 0,
        "sum_revenue": df_users["total_revenue"].sum() if len(df_users) else 0,
        "users_total": len(df_users),
        "users_with_purchase": (df_users["total_revenue"] > 0).sum()
    })

ts = pd.DataFrame(rows)
ts["day"] = pd.to_datetime(ts["day"])

fig = px.line(ts, x="day", y="sum_revenue",
              title="Общая выручка по дням")

    # период теста: [2022-03-23, 2022-03-30)
test_start = pd.to_datetime("2022-03-23")
test_end   = pd.to_datetime("2022-03-30")
fig.add_vline(x=test_start, line_dash="dash")
fig.add_vline(x=test_end, line_dash="dash")
fig.add_vrect(x0=test_start, x1=test_end, opacity=0.15, line_width=0)

fig.update_layout(xaxis_title="Дата", yaxis_title="Общая выручка")
fig.show()

In [20]:
fig = px.line(ts, x="day", y="users_with_purchase",
              title="Количество людей с покупками по дням")

    # период теста: [2022-03-23, 2022-03-30)
test_start = pd.to_datetime("2022-03-23")
test_end   = pd.to_datetime("2022-03-30")
fig.add_vline(x=test_start, line_dash="dash")
fig.add_vline(x=test_end, line_dash="dash")
fig.add_vrect(x0=test_start, x1=test_end, opacity=0.15, line_width=0)

fig.update_layout(xaxis_title="Дата", yaxis_title="Количество покупателей")
fig.show()